# TODO:

Add the llm verifier to check the thought process and see if this leads to a faster convergence

Fix exact match reward, also report as a bug on unsloth github

Create new dataset which uses the brain teasers I know

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [2]:
#@title Colab Extra Install { display-mode: "form" }
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    
    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [3]:
from unsloth import FastModel
import torch
max_seq_length = 1024

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = True, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-25 19:33:28 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-25 19:33:28 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


In [4]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [9]:
from datasets import load_dataset
#dataset = load_dataset("openai/gsm8k", "main", split = "train")
#dataset

# load dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

In [10]:

# load dataset from a json 
file_path = 'brain_teasers.json'
dataset = load_dataset(
    'json',
    data_files=file_path,
    split='train'
)




Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
reasoning_start = "<thinking>"
reasoning_end   = "</thinking>"
solution_start = "<response>"
solution_end = "</response>"

system_prompt = \
f"""You are given a world class reasoning large language model.
You are given a message below and return extremely valuable responses. To decide on that response, you must first *think* about what the user is asking you.

Place your thinking between {reasoning_start} and {reasoning_end}.
Then, provide your response between {solution_start}{solution_end}"""
print(system_prompt)


def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["answer"])

dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": extract_hash_answer(x["answer"]),
})
dataset[0]

You are given a problem.
Think about the problem and provide your working out.
Place it between <thinking> and </thinking>.
Then, provide your solution between <answer></answer>


Map:   0%|          | 0/6 [00:00<?, ? examples/s]

{'question': 'How many rs in strawberry',
 'answer': None,
 'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <thinking> and </thinking>.\nThen, provide your solution between <answer></answer>',
   'role': 'system'},
  {'content': 'How many rs in strawberry', 'role': 'user'}]}

In [12]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)


thoughts_match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)


In [13]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

In [14]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

In [15]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # Correct answer gets 3 points!
        if guess == true_answer:
            score += 3.0
        # Match if spaces are seen
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

In [16]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.,]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<answer>123.45</answer>")

['123.45']

In [17]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            guess       = float(guess.strip())
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0)
            continue
    return scores

In [18]:
temperature = 0.
from typing import Optional
def extract_xml(response: str, xml_tag: str) -> Optional[str]:
    """Extract content from XML tags in the response."""
    try:
        xml_start = f"<{xml_tag}>"
        xml_end = f"</{xml_tag}>"
        return response.split(xml_start)[1].split(xml_end)[0].replace("\n", " ").strip()
    except (IndexError, AttributeError):
        return None

from openai import OpenAI
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),

)

judge_model = "cohere/command-a"

def evaluate_thought_process(prompt, completion):
    evaluation_prompt = """You are a logical reasoning assessment tool designed to evaluate the quality of thought processes in AI-generated responses. Your task is to analyze a given response to a question and assign a score from 0 to 10 based on the logical soundness of the reasoning that led to the conclusion.

Scoring Criteria:
0-2: No thought process provided
3-4: Short thought process, and contains factual errors or is otherwise incorrect
5-6: Generally reasonable but contains factual errors or is otherwise incorrect
7-8: Sound logical structure with minor issues in reasoning or explanation
9-10: Similar process to how a human would answer the question, and contains no factual errors

Output Format:
Please provide your response within XML tags. First reply with the reasoning used to reach the result found, followed by the score as an integer between 0 and 10.

Example:
<reasoning>The reasoning is flawed because the conclusion does not logically follow from the premises.</reasoning>
<score>2</score>"""
    messages = [
        {"role": "system", "content": evaluation_prompt},
        {
            "role": "user",
            "content": f"Question: {prompt}\nAnswer: {completion}",
        },
    ]

    response = client.chat.completions.create(
                model=judge_model,
                messages=messages,
                temperature=temperature,
            )    

    score = float(
        extract_xml(response.choices[0].message.content, "score") or 0
    )
    reason = extract_xml(response.choices[0].message.content, "reasoning") or "No reasoning provided"
    return score, reason


def check_thought_process(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    scores = []
    reasons = []
    for prompt, completion in zip(prompts, responses):
        # Extract the thought process and score from the completion
        
        # Evaluate the thought process
        score, reason = evaluate_thought_process(prompt[-1]["content"], completion)
        reasons.append(reason)
        scores.append(score)

    print("CHECKING THOUGHTS")
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nScore:\n{scores[0]}", f"\nReason:\n{reasons[0]}")
    print("="*20)
    return scores

In [19]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        check_thought_process
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6 | Num Epochs = 9 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 6,522,880/1,006,408,832 (0.65% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 32768, 'top_k': 64, 'top_p': 0.95, 'bos_token_id': 2, 'eos_token_id': [1, 106]}. If this is not desired, please set these values explicitly.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


CHECKING THOUGHTS
******************** Question:
how many players have played for the nets and sixers 
Answer:
None 
Response:
<thinking>
This is a classic probability problem involving the number of players on each team. We need to determine how many players have played for both the Nets and the Sixers.  We'll use the principle of inclusion-exclusion.

Let N be the set of players on the Nets, and let S be the set of players on the Sixers. We want to find |N ∪ S|.

The total number of players on each team is not explicitly given. However, we can estimate the number of players on each team based on their current rosters. Let's assume the current rosters are complete and that the number of players on each team is reasonably high (e.g., 150-200 players).

The number of players on the Nets is |N| = 150.
The number of players on the Sixers is |S| = 150.

We want to find |N ∪ S|. Using the principle of inclusion-exclusion:

|N ∪ S| = |N| + |S| - |N ∩ S|

We don't have information about the i

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / check_thought_process
1,0.000000,3.500000,0.577350,265.000000,0.000000,3.500000
2,0.000000,5.250000,1.258306,230.250000,0.000000,5.250000
3,0.000000,3.750000,1.500000,433.750000,0.000541,3.750000
4,0.000000,4.750000,1.500000,277.500000,0.001051,4.750000
5,0.000000,5.500000,2.081666,409.500000,0.000646,5.500000
6,0.000100,8.250000,3.500000,70.750000,0.001281,8.250000
7,0.000100,6.000000,3.162278,55.500000,0.001833,6.000000
8,0.000000,5.000000,2.000000,318.500000,0.001139,5.000000
9,0.000000,4.000000,2.000000,423.000000,0.000787,4.000000
10,0.000100,3.000000,0.000000,154.750000,0.002257,3.000000


Unsloth: Will smartly offload gradients to save VRAM!
CHECKING THOUGHTS
******************** Question:
How many countries have a flag with exactly three colors 
Answer:
None 
Response:
<thinking>
The problem asks us to find the number of countries with flags that have exactly three colors. We need to think of all the countries and examine their flags.

Let's consider some common examples:
- China: Red, Yellow, Black
- Netherlands: Red, White, Blue
- Spain: Red, Yellow, Blue
- Brazil: Green, Yellow, Red
- Egypt: Red, White, Brown
- Japan: Red, Blue, Black
- South Korea: Red, Yellow, Green

We are looking for countries with a flag that contains three distinct colors.

- China: Red, Yellow, Black
- Netherlands: Red, White, Blue
- Spain: Red, Yellow, Blue
- Brazil: Green, Yellow, Red
- Egypt: Red, White, Brown
- Japan: Red, Blue, Black
- South Korea: Red, Yellow, Green

We have identified the following countries with flags that have exactly three colors:
- China
- Netherlands
- Spain
- Bra

In [ ]:
inference_question = "How many presidents were born in one of the thirteen original colonies?"# "How many pieces in chess start on a black tile and can move on the first turn?"
inference_prompt = [{"role": "system", "content": system_prompt}, {"role": "user", "content": inference_question}]

prompt = tokenizer.apply_chat_template(
    inference_prompt,
    add_generation_prompt = True,
    tokenize = False,
)

input_ids = tokenizer.encode(prompt, return_tensors = "pt")
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)

print(output_text)

In [ ]:
print(system_prompt)

In [ ]:
inference_question = "How many pieces in chess start on a black tile and can move on the first turn?"
inference_prompt = [{"role": "system", "content": system_prompt}, {"role": "user", "content": inference_question}]

In [ ]:
# using this prompt and result, evaluate thought process

